In [1]:
#include <string>       // std::string
#include <iostream>     // std::cout
#include <sstream>      // std::ostringstream
#include <fstream>

#include <vector>
#include <cmath>
#include <math.h>
#include <cstdlib>
#include <map>
#include "TH1.h"
#include "TH2.h"
#include "TProfile.h"
#include "TCanvas.h"
#include "TGraph.h"
#include "TFile.h"
#include "TString.h"
#include "TStyle.h"


using namespace std;

In [2]:
// Some chars
char myfname[100];
char myhname[100];
char myhfit[100];
char title_low[5000];

// Status box configurations
gStyle->SetOptStat(0);
gStyle->SetOptFit(101111);
gStyle->SetStatY(0.9);   // Set y-position (fraction of pad size)
gStyle->SetStatX(0.9);   // Set x-position (fraction of pad size)
gStyle->SetStatW(0.19);  // Set width of stat-box (fraction of pad size)
gStyle->SetStatH(0.15);  // Set height of stat-box (fraction of pad size)

// Input file
TFile *inputfile = TFile::Open("../results/htwInterpPix.root");

// Output files
// Output 1: root file with fits
sprintf(myfname,"../results/twFits.root");
TFile *outfile = new TFile(myfname, "RECREATE");

// Fit configurations
cout << " --------------------------- LOWER CURVE: Fitting --------------------------------- " << endl;

// Output 2: txt file with the values of the parameters in each column
string textfilename = "../results/loweredge_parameters.txt";
std::ostringstream ostr;
ostr << textfilename ;
std::ofstream outfile_low;
outfile_low.open (ostr.str().c_str(), std::ofstream::out);
outfile_low << "#row \tcol \tt0 \tt \tdeltaT \tVTP_t \tpar0 \tpar1 \tpar2 \tpar3" << endl;

cout << textfilename << endl;

cout << " LOWER CURVE: Fitting ..... " << endl;
cout << "Be patient, this might take a while" << endl << endl;;

// It can be done for all 256 row and 256 columns of the matrix, just change the loop to irow++/icol++. 
// Be aware that analyse the entire pixel matrix can take a lot of RAM memory and the notebook can crash
for(int irow=0; irow<256; irow+=16){     // sweep through rows of the pixel array
    for(int icol=0; icol<256; icol+=16){ // sweep through columns of the pixel array
        
        TH2F *htw_low=NULL;

        // Get histograms from input file
        sprintf(myhname,"htwAllperpix_r%d_c%d_lowEdge", irow, icol);
        htw_low =  (TH2F*) inputfile->Get(myhname);

        // Fit configurations
        sprintf(title_low,"Plot for pixel %dx%d - Lower Edge; |#Delta VTP|; BXID+(1.0/16)*(15-phase)  [ns]", irow, icol);
        htw_low->SetTitle(title_low);
        TString funcname = "fit";
        TString func = ""; func+=funcname; func+=irow; func+="_c"; func+=icol;
        TString lowerfuncname = func+"_lowcurve";
        
        cout << lowerfuncname << endl;

        // Initial parameters
        double initx   = htw_low->GetBinCenter(htw_low->GetMaximumBin()); // Get maximun bin for the x range
        double lastx   = htw_low->FindLastBinAbove(0,1); // Get last bin with content > 0 for the x range
        double maxx    = htw_low->GetBinCenter(htw_low->GetMaximumBin()); // Get maximun bin to set fit parameters
        double maxy    = htw_low->GetBinContent(htw_low->GetMaximumBin()); // Return content of bin number bin to set fi parameters
        double t_thl_l = initx;

        cout << "X axis range = [" << initx << ", " << lastx << "]" << endl;
        cout << "Maximun bin = " << maxx << endl;
        cout << "Content of maximun bin = " << maxy << endl;

        double par3_hint_bin = htw_low->FindBin(lastx);
        double par3_hint     = htw_low->GetBinContent(par3_hint_bin);

        // Define fuction
        TF1 *lowerfunc = new TF1(lowerfuncname,"([0]/(x-[1])**[2]) + [3]" , 0, 510);

        // Set fit parameter values
        lowerfunc->SetParameters(100, maxx-1, 0.9, par3_hint);
        lowerfunc->SetParLimits(0, 0, 4000);
        lowerfunc->SetParLimits(1, -20, 20);
        lowerfunc->SetParLimits(2, 0.1, 4);
        lowerfunc->SetLineWidth(2);
        lowerfunc->SetLineStyle(1);
        lowerfunc->SetLineColor(4);
        lowerfunc->SetMaximum(maxy);
        lowerfunc->SetRange(initx-1, lastx);
        
        // Fit function
        sprintf(myhfit,"fit_r%d_c%d_lowEdge", irow, icol);
        TCanvas *canvas = new TCanvas(myhfit,myhfit,800,600);
        htw_low->SetMarkerSize(0.6);
        htw_low->GetYaxis()->SetRangeUser(600, 680);
        
        htw_low->Fit(lowerfuncname, "MR0W", "", initx+1, lastx);
        htw_low->Draw("p");
        lowerfunc->Draw("same");
        
        gStyle->SetOptFit(101111);
        gStyle->SetOptStat(0);
        
        // Save fit for pixel (0,0) to put on the paper
        if(irow==0 && icol==0){
            canvas->SaveAs("../deliver/images/low_edge_fit_0x0.png");
        }
        
        // Save plots
        //htw_low->Write();   // Save histogram individually
        //lowerfunc->Write(); // Save function individually
        canvas->Update();
        canvas->Write();
        canvas->Clear();

        // After fitting, save the resulted parameter values into variables
        double par0_l = lowerfunc->GetParameter(0);
        double par1_l = lowerfunc->GetParameter(1);
        double par2_l = lowerfunc->GetParameter(2);
        double par3_l = lowerfunc->GetParameter(3);

        double t0_bin_l = htw_low->FindBin(510);
        double t0_l = htw_low->GetBinContent(t0_bin_l);
        cout << "Time for infinite charge = " << t0_l << endl;

        // Calculate minimum charge for Timewalk effect 
        bool check_l = false;
        for(Double_t icharge = 100; icharge > 0; icharge-=0.0001){
            Double_t t_func = (par0_l/pow(icharge-par1_l,par2_l))+par3_l;
            Double_t deltaT = t_func - par3_l;
            Double_t VTP_t = icharge;

            //cout << "VTP = " << icharge << ", deltaT = " << deltaT << endl; 

            if(deltaT > 25){
                while(check_l != true){
                //cout << "VTP (bin " << t_bin_l << ") = " << icharge << ", t = " << t_l << ", deltaT = " << deltaT <<  endl; //break;
                cout << "VTP (" << t_func << ") = " << icharge << ", t = " << t_func << ", deltaT = " << deltaT <<  endl; //break;

                outfile_low << irow << "\t" << icol << "\t" << t0_l << "\t" << t_func << "\t" << deltaT << "\t" << VTP_t << "\t" << par0_l << "\t" <<  par1_l << "\t" << par2_l << "\t" << par3_l << endl;

                check_l = true;
                    
                }
            }
        }

        cout << "sucess" << endl;

        if(check_l !=true){
            for(Double_t icharge = 0; icharge <= 50; icharge+=0.0001){
            Double_t t_func = (par0_l/pow(icharge-par1_l,par2_l))+par3_l;
            Double_t deltaT = t_func - par3_l;
            Double_t VTP_t = icharge;

            //cout << "VTP = " << icharge << ", deltaT = " << deltaT << endl;

                if(deltaT > 24.9 && deltaT <= 25){
                    while(check_l != true){
                    cout << "VTP (" << t_func << ") = " << icharge << ", t = " << t_func << ", deltaT = " << deltaT <<  endl; //break;

                    outfile_low << irow << "\t" << icol << "\t" << t0_l << "\t" << t_func << "\t" << deltaT << "\t" << VTP_t << "\t" << par0_l << "\t" <<  par1_l << "\t" << par2_l << "\t" << par3_l << endl;
                    check_l = true;
                        
                    }
                }
            }
        }
        
    
    } // icol loop
} // irow loop

cout << endl << " All indicated pixels analysed " << endl;

// Close everything
outfile_low.close();
inputfile->Close();
outfile->Close();

 --------------------------- LOWER CURVE: Fitting --------------------------------- 
../results/loweredge_parameters.txt
 LOWER CURVE: Fitting ..... 
Be patient, this might take a while

fit0_c0_lowcurve
X axis range = [18, 511]
Maximun bin = 18
Content of maximun bin = 662.496
 FCN=508.949 FROM HESSE     STATUS=OK             23 CALLS         822 TOTAL
                     EDM=1.3637e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.41014e+02   3.08400e+01   6.21332e-06   2.31883e-01
   2  p1           1.53289e+01   6.54543e-01   2.03233e-05   1.35355e-01
   3  p2           8.77258e-01   7.13238e-02   7.97614e-06  -6.99361e-02
   4  p3           6.08853e+02   2.51993e-01   1.81888e-04   2.40032e-03
Time for infinite charge = 608.832
VTP (633.853) = 22.5142, t = 633.853, deltaT = 25.0001
sucess
fit0_c16_lowcurve
X axis range =

Info in <TCanvas::Print>: png file ../deliver/images/low_edge_fit_0x0.png has been created


VTP (630.978) = 17.7595, t = 630.978, deltaT = 25.0002
sucess
fit0_c32_lowcurve
X axis range = [16, 511]
Maximun bin = 16
Content of maximun bin = 656.223
 FCN=21.4343 FROM HESSE     STATUS=OK             23 CALLS         572 TOTAL
                     EDM=3.23988e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           6.93944e+01   2.37539e+01   4.59225e-06  -1.57215e-02
   2  p1           1.10586e+01   2.20010e+00   5.02469e-05   1.62137e-02
   3  p2           5.94221e-01   1.06857e-01   7.35893e-06   9.39406e-03
   4  p3           6.06271e+02   6.84559e-01   2.89092e-04   9.66376e-04
Time for infinite charge = 608.348
VTP (631.271) = 16.6325, t = 631.271, deltaT = 25
sucess
fit0_c48_lowcurve
X axis range = [16, 511]
Maximun bin = 16
Content of maximun bin = 657.824
 FCN=21.1881 FROM HESSE     STATUS=OK             23 CALLS         

VTP (630.186) = 19.1731, t = 630.186, deltaT = 25.0002
sucess
fit0_c224_lowcurve
X axis range = [16, 511]
Maximun bin = 16
Content of maximun bin = 654.676
 FCN=28.1388 FROM HESSE     STATUS=OK             23 CALLS         886 TOTAL
                     EDM=1.57662e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.06901e+01   8.93249e-01   2.16340e-06  -1.78233e-02
   2  p1           1.79058e+01   3.96668e-02   2.17566e-06  -6.82936e-02
   3  p2           2.38077e-01   3.23712e-02   6.48451e-06  -3.94395e-02
   4  p3           6.00747e+02   1.59182e+00   2.86458e-04   9.11766e-04
Time for infinite charge = 608.348
VTP (625.747) = 20.2721, t = 625.747, deltaT = 25
sucess
fit0_c240_lowcurve
X axis range = [14, 511]
Maximun bin = 14
Content of maximun bin = 664.059
 FCN=23.8418 FROM HESSE     STATUS=OK             23 CALLS       

sucess
fit16_c160_lowcurve
X axis range = [14, 511]
Maximun bin = 14
Content of maximun bin = 660.973
 FCN=31.4193 FROM HESSE     STATUS=OK             23 CALLS         777 TOTAL
                     EDM=1.26531e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.32665e+01   9.70913e-01   1.07998e-06  -6.14592e-01
   2  p1           1.58209e+01   5.49929e-02   4.97472e-07   2.22738e-01
   3  p2           2.95808e-01   3.07883e-02   1.41994e-06  -1.15440e-01
   4  p3           6.02408e+02   1.05019e+00   1.43625e-04   4.19554e-03
Time for infinite charge = 608.348
VTP (627.408) = 18.4476, t = 627.408, deltaT = 25.0002
sucess
fit16_c176_lowcurve
X axis range = [20, 511]
Maximun bin = 20
Content of maximun bin = 654.652
 FCN=18.2366 FROM HESSE     STATUS=OK             23 CALLS         719 TOTAL
                     EDM=9.45014e-08

...

Info in ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 


VTP (633.666) = 14.5483, t = 633.666, deltaT = 25.0002
sucess
fit208_c80_lowcurve
X axis range = [12, 511]
Maximun bin = 12
Content of maximun bin = 659.371
 FCN=21.4048 FROM HESSE     STATUS=OK             23 CALLS         732 TOTAL
                     EDM=1.76614e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.34133e+01   2.17731e+00   1.16716e-06   1.56677e-01
   2  p1           1.32969e+01   1.98788e-01   1.51430e-06   4.89558e-02
   3  p2           3.72079e-01   4.36752e-02   1.47447e-06  -2.72423e-02
   4  p3           6.05666e+02   8.20926e-01   1.44402e-04   1.26109e-03
Time for infinite charge = 608.66
VTP (630.666) = 15.4775, t = 630.666, deltaT = 25.0002
sucess
fit208_c96_lowcurve
X axis range = [12, 511]
Maximun bin = 12
Content of maximun bin = 657.84
 FCN=329.232 FROM HESSE     STATUS=OK             23 CALLS  

VTP (635.615) = 14.993, t = 635.615, deltaT = 25
sucess
fit224_c16_lowcurve
X axis range = [18, 511]
Maximun bin = 18
Content of maximun bin = 667.16
 FCN=25.2033 FROM HESSE     STATUS=OK             23 CALLS         892 TOTAL
                     EDM=3.50399e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.31527e+02   7.58629e+01   1.69513e-06  -8.17958e-02
   2  p1           1.07029e+01   3.12299e+00   1.04752e-05   6.99773e-03
   3  p2           8.22773e-01   1.56036e-01   1.81953e-06   1.16913e-01
   4  p3           6.10343e+02   4.35153e-01   1.45517e-04  -4.83242e-03
Time for infinite charge = 611.496
VTP (635.344) = 18.226, t = 635.344, deltaT = 25.0001
sucess
fit224_c32_lowcurve
X axis range = [14, 511]
Maximun bin = 14
Content of maximun bin = 657.84
 FCN=313.152 FROM HESSE     STATUS=OK             23 CALLS         

 FCN=24.7146 FROM HESSE     STATUS=OK             23 CALLS         760 TOTAL
                     EDM=1.26686e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.50272e+01   2.30673e+00   2.64757e-06   8.63083e-02
   2  p1           1.53129e+01   1.80166e-01   8.24451e-06  -4.75831e-02
   3  p2           3.91273e-01   4.16641e-02   8.00251e-06   6.07690e-04
   4  p3           6.06164e+02   7.27798e-01   2.89041e-04  -2.33555e-04
Time for infinite charge = 609.191
VTP (631.164) = 17.6806, t = 631.164, deltaT = 25.0002
sucess
fit224_c208_lowcurve
X axis range = [14, 511]
Maximun bin = 14
Content of maximun bin = 660.965
 FCN=16.7133 FROM HESSE     STATUS=OK             23 CALLS         939 TOTAL
                     EDM=9.1229e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP    

VTP (630.542) = 16.7815, t = 630.542, deltaT = 25.0002
sucess
fit240_c128_lowcurve
X axis range = [16, 511]
Maximun bin = 16
Content of maximun bin = 657.84
 FCN=19.8103 FROM HESSE     STATUS=OK             23 CALLS        1098 TOTAL
                     EDM=9.83985e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.07815e+02   7.12347e+01   1.46689e-06   6.28420e-02
   2  p1           7.84375e+00   3.94422e+00   1.10984e-05  -1.04784e-02
   3  p2           7.74303e-01   1.79583e-01   1.71022e-06  -1.10448e-01
   4  p3           6.09162e+02   5.32450e-01   1.45236e-04   6.46391e-03
Time for infinite charge = 609.926
VTP (634.162) = 14.447, t = 634.162, deltaT = 25
sucess
fit240_c144_lowcurve
X axis range = [16, 511]
Maximun bin = 16
Content of maximun bin = 654.668
 FCN=835.088 FROM HESSE     STATUS=OK             23 CALLS     